In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

In [0]:
def generate_time_series(batch_size, n_steps):
  freq1, freq2, offsets1, offsets2 = np.random.rand(4,batch_size,1)
  time = np.linspace(0,1,n_steps)
  series = 0.5 * np.sin((time - offsets1)*(freq1*10+10))
  series += 0.2 * np.sin((time - offsets2)*(freq2*20+20))
  series += 0.1 * (np.random.rand(batch_size, n_steps)-0.5)
  return series[...,np.newaxis].astype(np.float32)

In [0]:
n_steps = 50
series = generate_time_series(10000,n_steps+10)
X_train, y_train = series[:7000,:n_steps], series[:7000,-10:,0]
X_valid,y_valid = series[7000:9000,:n_steps], series[7000:9000,-10:,0]
X_test,y_test = series[9000:,:n_steps], series[9000:,-10:,0]

In [0]:
Y = np.empty((10000,n_steps,10))
for step_ahead in range(1,10+1):
  Y[:,:,step_ahead-1] = series[:,step_ahead:step_ahead+n_steps, 0]
Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]

# Wavenet

In [0]:
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=[None,1]))
for rate in (1,2,4,8) * 2:
  model.add(keras.layers.Conv1D(filters=20,kernel_size=2,padding='causal',activation='relu',dilation_rate=rate))
model.add(keras.layers.Conv1D(filters=10,kernel_size=1))

In [0]:
def last_time_step(Y_true,Y_pred):
  return keras.metrics.mean_squared_error(Y_true[:,-1],Y_pred[:,-1])

In [0]:
model.compile(
    loss = "mse",
    optimizer="adam",
    metrics = [last_time_step]
)

In [12]:
model.fit(X_train,Y_train,epochs=20,validation_data=(X_valid,Y_valid))

Epoch 1/20
219/219 [==============================] - 4s 17ms/step - loss: 0.0731 - last_time_step: 0.0626 - val_loss: 0.0386 - val_last_time_step: 0.0249
Epoch 2/20
219/219 [==============================] - 3s 14ms/step - loss: 0.0349 - last_time_step: 0.0217 - val_loss: 0.0319 - val_last_time_step: 0.0186
Epoch 3/20
219/219 [==============================] - 3s 14ms/step - loss: 0.0304 - last_time_step: 0.0177 - val_loss: 0.0287 - val_last_time_step: 0.0152
Epoch 4/20
219/219 [==============================] - 3s 15ms/step - loss: 0.0280 - last_time_step: 0.0154 - val_loss: 0.0273 - val_last_time_step: 0.0143
Epoch 5/20
219/219 [==============================] - 3s 14ms/step - loss: 0.0267 - last_time_step: 0.0142 - val_loss: 0.0264 - val_last_time_step: 0.0136
Epoch 6/20
219/219 [==============================] - 3s 15ms/step - loss: 0.0256 - last_time_step: 0.0132 - val_loss: 0.0251 - val_last_time_step: 0.0123
Epoch 7/20
219/219 [==============================] - 3s 14ms/step - l